In [109]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets seqeval
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pyyaml>=5.1
!pip install torch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import LayoutLMv2Tokenizer, LayoutLMv2ForTokenClassification, LayoutLMv2Config
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import ImageDraw, ImageFont, Image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings 
warnings.filterwarnings('ignore')

In [111]:
# from urllib2 import Request, urlopen
import json
import pandas as pd

# reading the JSON data using json.load()
file = '/content/2.json'

df = pd.read_json(file)

In [112]:
df.head()

,ocr,id,bbox,label,transcription,annotator,annotation_id,created_at,updated_at,lead_time
0,/content/data/1.png,1,"[[15.466666666666667, 15.457115928369463, 35.4...","[produttore, Unit.Loc., Orario_richiesta, Data...","[ SPAZIO SOC COO, Via S. Agostino, 2 51100 PIS...",1,1,2022-06-14 09:57:19.654695+00:00,2022-06-14 16:07:17.971902+00:00,25734.315
1,/content/data/2.png,4,"[[15.466666666666667, 15.174363807728557, 35.6...","[produttore, Unit.Loc., Data_richiesta, Orario...","[ALSTOM FERROVIA SPA, FIRENZE -Via del Romito ...",1,4,2022-06-14 11:36:49.515327+00:00,2022-06-14 12:43:02.012909+00:00,13478.754


In [113]:
dropped = ['annotator','annotation_id','created_at','updated_at','lead_time']
data = df.drop(dropped,axis=1)
data

,ocr,id,bbox,label,transcription
0,/content/data/1.png,1,"[[15.466666666666667, 15.457115928369463, 35.4...","[produttore, Unit.Loc., Orario_richiesta, Data...","[ SPAZIO SOC COO, Via S. Agostino, 2 51100 PIS..."
1,/content/data/2.png,4,"[[15.466666666666667, 15.174363807728557, 35.6...","[produttore, Unit.Loc., Data_richiesta, Orario...","[ALSTOM FERROVIA SPA, FIRENZE -Via del Romito ..."


In [114]:
data['imageWidth'] = '1901'
data['imageHeight'] = '2690'

In [115]:
data.rename(columns = {'ocr':'image_path', 'transcription':'words'}, inplace = True)

  #  "original_width": 1901,
  #       "original_height": 2690

In [116]:
data

,image_path,id,bbox,label,words,imageWidth,imageHeight
0,/content/data/1.png,1,"[[15.466666666666667, 15.457115928369463, 35.4...","[produttore, Unit.Loc., Orario_richiesta, Data...","[ SPAZIO SOC COO, Via S. Agostino, 2 51100 PIS...",1901,2690
1,/content/data/2.png,4,"[[15.466666666666667, 15.174363807728557, 35.6...","[produttore, Unit.Loc., Data_richiesta, Orario...","[ALSTOM FERROVIA SPA, FIRENZE -Via del Romito ...",1901,2690


In [117]:
type(data.words)  

pandas.core.series.Series

In [118]:
type(data.words[0][0])

str

In [119]:
data
data.to_pickle("/content/data.pkl")

In [120]:
# data.to_csv('/content/data.csv')

In [121]:
# data=pd.read_csv('data.csv')

In [122]:
data['bbox']

0    [[15.466666666666667, 15.457115928369463, 35.4...
1    [[15.466666666666667, 15.174363807728557, 35.6...
Name: bbox, dtype: object

In [123]:
class data_config:
    labels = np.unique([item for sublist in data.label for item in sublist]).tolist()
    #labels = sum([["B-" + item, "I-" + item] for item in np.unique(labels)], [])
    labels = ["B-" + item for item in np.unique(labels)]
    num_labels = len(labels)
    id2label = {v: k for v, k in enumerate(labels)}
    label2id = {k: v for v, k in enumerate(labels)}

In [124]:
model_path = 'microsoft/layoutlmv2-base-uncased'
config = LayoutLMv2Config.from_pretrained(model_path, num_labels=data_config.num_labels, id2label = data_config.id2label, label2id = data_config.label2id)
tokenizer = LayoutLMv2Tokenizer.from_pretrained(model_path)
model = LayoutLMv2ForTokenClassification.from_pretrained(model_path, config = config)
model.to(device)

Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res5.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.21.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.17.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.2.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.0.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.21.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.0.conv2.norm.num_batches_tracked'

LayoutLMv2ForTokenClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [125]:
PATH='/content/'
torch.save(model.state_dict(), PATH +'layoutlm_EuroCoorp.pt')

In [130]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import LayoutLMv2Tokenizer, LayoutLMv2ForTokenClassification, LayoutLMv2Config
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import ImageDraw, ImageFont, Image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings 
warnings.filterwarnings('ignore')

def normalize_box(box, width, height):
    width = int(width)
    height = int(height)
    # [ <class 'str'> 1901 <class 'int'>
    # print("YYYY: ", box,'*',type(box[0]),'*',  width,'*', type(width) ,'*')
    return [
         int(1000 * (int(box[0]) / width)),
         int(1000 * (int(box[1]) / height)),
         int(1000 * (int(box[2]) / width)),
         int(1000 * (int(box[3]) / height)),
     ]

def de_normalize(box, width, height):
  return [
          int((width * box[0])/1000),
          int((height * box[1])/1000),
          int((width * box[2])/1000),
          int((height * box[3])/1000)
  ]

def resize_and_align_bounding_box(bbox, original_image, target_size):
    x_, y_ = original_image.size
    x_scale = target_size / x_ 
    y_scale = target_size / y_
    origLeft, origTop, origRight, origBottom = tuple(bbox)
    x = int(np.round(origLeft * x_scale))
    y = int(np.round(origTop * y_scale))
    xmax = int(np.round(origRight * x_scale))
    ymax = int(np.round(origBottom * y_scale)) 
    return [x-0.5, y-0.5, xmax+0.5, ymax+0.5]
 
class InvoiceDataSet(Dataset):
    """LayoutLM dataset with visual features."""
 
    def __init__(self, df, tokenizer, max_length, target_size, train=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_seq_length = max_length
        self.target_size = target_size
        self.pad_token_box = [0, 0, 0, 0]
        self.train = train
 
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        item = self.df.iloc[idx,:].to_dict()        
        #base_path = data_config.base_image_path
        original_image = Image.open(os.path.join('/content/data/' , item["image_path"])).convert("RGB")
        # resize to target size (to be provided to the pre-trained backbone)
        resized_image = original_image.resize((self.target_size, self.target_size))
        # first, read in annotations at word-level (words, bounding boxes, labels)
        words = item["words"]
        # print(words)
        # print("IN GETITEM XXX: ", type(words), type(words[0]))
        unnormalized_word_boxes = item["bbox"]
        # print(unnormalized_word_boxes)

        # print("IN boxes XXX: ", type(unnormalized_word_boxes), type(unnormalized_word_boxes[0]))
        word_labels = item["label"]
        # print(word_labels)

        width = item["imageWidth"]
        height = item["imageHeight"]
        normalized_word_boxes = [normalize_box(bbox, width, height) for bbox in unnormalized_word_boxes]
        # print('normalized_word_boxes',normalized_word_boxes)
        assert len(words) == len(normalized_word_boxes)
 
        # next, transform to token-level (input_ids, attention_mask, token_type_ids, bbox, labels)
        token_boxes = []
        unnormalized_token_boxes = []
        token_labels = []
        for word, unnormalized_box, box, label in zip(words, unnormalized_word_boxes, normalized_word_boxes, word_labels):
            word_tokens = self.tokenizer.tokenize(word)
            unnormalized_token_boxes.extend(unnormalized_box for _ in range(len(word_tokens)))
            token_boxes.extend(box for _ in range(len(word_tokens)))
            # label first token as B-label (beginning), label all remaining tokens as I-label (inside)
            for i in range(len(word_tokens)):
                if  1 == 1:#i == 0:
                    token_labels.extend(['B-' + label])
                else:
                    token_labels.extend(['I-' + label])
        
        # Truncation of token_boxes + token_labels
        special_tokens_count = 2 
        if len(token_boxes) > self.max_seq_length - special_tokens_count:
            token_boxes = token_boxes[: (self.max_seq_length - special_tokens_count)]
            unnormalized_token_boxes = unnormalized_token_boxes[: (self.max_seq_length - special_tokens_count)]
            token_labels = token_labels[: (self.max_seq_length - special_tokens_count)]
        
        # add bounding boxes and labels of cls + sep tokens
        token_boxes = [self.pad_token_box] + token_boxes + [[1000, 1000, 1000, 1000]]
        unnormalized_token_boxes = [self.pad_token_box] + unnormalized_token_boxes + [[1000, 1000, 1000, 1000]]
        token_labels = [-100] + token_labels + [-100]
        print('token_labels',token_labels)
        # return token_labels

        encoding = self.tokenizer(' '.join(words), padding='max_length', truncation=True)
        # Padding of token_boxes up the bounding boxes to the sequence length.
        print('encoding',encoding)
        input_ids = self.tokenizer(' '.join(words), truncation=True)["input_ids"]
        padding_length = self.max_seq_length - len(input_ids)
        token_boxes += [self.pad_token_box] * padding_length
        unnormalized_token_boxes += [self.pad_token_box] * padding_length
        token_labels += [-100] * padding_length
        encoding['bbox'] = token_boxes
        encoding['labels'] = token_labels
 
        assert len(encoding['input_ids']) == self.max_seq_length
        assert len(encoding['attention_mask']) == self.max_seq_length
        assert len(encoding['token_type_ids']) == self.max_seq_length
        assert len(encoding['bbox']) == self.max_seq_length
        assert len(encoding['labels']) == self.max_seq_length
 
        encoding['resized_image'] = ToTensor()(resized_image)
        # rescale and align the bounding boxes to match the resized image size (typically 224x224) 
        encoding['resized_and_aligned_bounding_boxes'] = [resize_and_align_bounding_box(bbox, original_image, self.target_size) for bbox in unnormalized_token_boxes]
        #encoding['unnormalized_token_boxes'] = unnormalized_token_boxes
        
        # finally, convert everything to PyTorch tensors 
        for k,v in encoding.items():
            if k == 'labels':
                label_indices = []
                # convert labels from string to indices
                for label in encoding[k]:
                    if label != -100:
                        label_indices.append(data_config.label2id[label])
                    else:
                        label_indices.append(label)
                encoding[k] = label_indices
            encoding[k] = torch.as_tensor(encoding[k])
        print(encoding)
        return encoding

In [131]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(data, test_size = 0.2)

train_dataset = InvoiceDataSet(df = train, tokenizer = tokenizer, max_length = 512, target_size = 224, train=True)
train_dataloader = DataLoader(train_dataset, batch_size=5)

valid_dataset = InvoiceDataSet(df = valid, tokenizer = tokenizer, max_length = 512, target_size = 224, train=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=5)

In [132]:
from transformers import AdamW
from tqdm.notebook import tqdm
import numpy as np
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
import torch


def train_fn(train_dataloader, model, optimizer):
    tk0 = tqdm(train_dataloader, total = len(train_dataloader))
    for bi, batch in enumerate(tk0):
        input_ids=batch['input_ids'].to(device)
        bbox=batch['bbox'].to(device)
        attention_mask=batch['attention_mask'].to(device)
        token_type_ids=batch['token_type_ids'].to(device)
        labels=batch['labels'].to(device)
        resized_images = batch['resized_image'].to(device) 
        resized_and_aligned_bounding_boxes = batch['resized_and_aligned_bounding_boxes'].to(device) 
        outputs = model(image = resized_images,input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
def eval_fn(eval_dataloader, model):
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    model.eval()
    tk0 = tqdm(eval_dataloader, total = len(eval_dataloader))
    for bi, batch in enumerate(tk0):
        with torch.no_grad():
            input_ids=batch['input_ids'].to(device)
            bbox=batch['bbox'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            token_type_ids=batch['token_type_ids'].to(device)
            labels=batch['labels'].to(device)
            resized_images = batch['resized_image'].to(device) 
            resized_and_aligned_bounding_boxes = batch['resized_and_aligned_bounding_boxes'].to(device)
            outputs = model(image = resized_images,input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
            tmp_eval_loss = outputs.loss
            logits = outputs.logits
            eval_loss += tmp_eval_loss.item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = labels.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(
                    out_label_ids, labels.detach().cpu().numpy(), axis=0
                )
    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=2)
    out_label_list = [[] for _ in range(out_label_ids.shape[0])]
    preds_list = [[] for _ in range(out_label_ids.shape[0])]
    for i in range(out_label_ids.shape[0]):
        for j in range(out_label_ids.shape[1]):
            if out_label_ids[i, j] != -100:
                out_label_list[i].append(config.id2label[out_label_ids[i][j]])
                preds_list[i].append(config.id2label[preds[i][j]])

    results = {
        "loss": eval_loss,
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }
    return results

In [133]:
MODEL_PATH ="/content/layoutlm_EuroCoorp.pt"
optimizer = AdamW(model.parameters(), lr=5e-5)
global_step = 0
best_f1_score = 0
for epoch in range(5):
    train_fn(train_dataloader, model, optimizer)
    current_f1_score = eval_fn(valid_dataloader, model)
    if current_f1_score["f1"] > best_f1_score:
        torch.save(model.state_dict(), MODEL_PATH)
        best_f1_score = current_f1_score["f1"]
    print("best_f1_score :", best_f1_score)

  0%|          | 0/1 [00:00<?, ?it/s]

token_labels [-100, 'B-produttore', 'B-produttore', 'B-produttore', 'B-produttore', 'B-produttore', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Unit.Loc.', 'B-Orario_richiesta', 'B-Orario_richiesta', 'B-Orario_richiesta', 'B-Data_richiesta', 'B-Data_richiesta', 'B-Data_richiesta', 'B-Data_richiesta', 'B-Data_richiesta', 'B-Data_richiesta', 'B-Cer', 'B-Cer', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Descrizone', 'B-Sf', 'B-Sf', 'B-Sf', 'B-Sf', 'B-Sf', 'B-Sf', 'B-Sf', 'B-Classe_Pericolo', 'B-Destino', 'B-Destino', 'B-Others', 'B-Ot

ValueError: ignored